In [1]:
import pandas as pd

In [2]:
# API КЛЮЧ 2GIS

#API_KEY = 'aba7c65b-986e-4352-aaef-013eb72c7a18' -- demo
API_KEY = 'd819a1bc-f820-4289-9562-04a5d2d18bf9'
API_URL = 'https://catalog.api.2gis.com/3.0/items'

# Настройки для Санкт-Петербурга
REGION_ID = 38  # ID Санкт-Петербурга

print("API ключ подгружен")

API ключ подгружен


In [3]:
# Ищем по ключевым словам

keywords = [
    # Теннис
    "теннисный корт",

    # Сквош
    "сквош корт",

    # Падел
    "падел",
]

print(f"Всего ключевых слов: {len(keywords)}")

Всего ключевых слов: 3


In [4]:
def search_objects(keyword):
    """
    Поиск ВСЕХ объектов через пагинацию с search_type='discovery'.
    Использует page_size=50 для каждого запроса.
    """
    print(f"Ищем: '{keyword}'")

    basic_fields = [
        'items.id',
        'items.name',
        'items.type',
        'items.point',
        'items.address',
        'items.adm_div',
        'items.address_name',
        'items.full_address_name',
        'items.rubrics',
        'items.name_ex',
        'items.caption',
        'items.region_id'
    ]

    # 1. Получаем общее количество объектов через discovery
    params_info = {
        'key': API_KEY,
        'q': keyword,
        'region_id': REGION_ID,
        'search_type': 'discovery',
        'page_size': 1,
        'fields': 'items.id,result.total'
    }

    try:
        print("Получаем информацию через discovery...")
        response = requests.get(API_URL, params=params_info, timeout=30)

        if response.status_code == 200:
            data = response.json()
            total = data.get('result', {}).get('total', 0)

            if total == 0:
                print("Объектов не найдено")
                return []

            print(f"Всего объектов по API: {total}")

            # 2. Загружаем объекты постранично с тем же search_type='discovery'
            all_objects = []
            page_size = 50  # Максимум для discovery, с которым работает
            offset = 0
            pages_estimated = (total + page_size - 1) // page_size  # Округление вверх

            for page in range(pages_estimated):
                params_load = {
                    'key': API_KEY,
                    'q': keyword,
                    'region_id': REGION_ID,
                    'search_type': 'discovery',  # Ключевой момент: ОСТАВЛЯЕМ тот же search_type
                    'page_size': min(page_size, total - offset),  # Для последней страницы
                    'offset': offset,
                    'fields': ','.join(basic_fields)
                }

                print(f"Запрос страницы {page+1}/{pages_estimated} (offset={offset})...")
                response = requests.get(API_URL, params=params_load, timeout=30)

                if response.status_code == 200:
                    data = response.json()
                    objects = data.get('result', {}).get('items', [])

                    if objects:
                        all_objects.extend(objects)
                        offset += len(objects)
                        print(f"Добавлено {len(objects)} объектов")
                    else:
                        print(f"Страница {page+1}: объектов не получено.")
                        # Прерываем цикл, если API перестал возвращать данные
                        break

                else:
                    print(f"Ошибка на странице {page+1}: {response.status_code}")
                    break

                # Короткая пауза между запросами
                time.sleep(0.2)

            print(f"Итого собрано: {len(all_objects)} объектов")
            return all_objects

        else:
            print(f"Ошибка начального запроса: {response.status_code}")

    except Exception as e:
        print(f"Ошибка: {e}")

    return []

In [5]:
# Функция для поиска рубрик

from typing import List, Dict

def search_rubrics(search_query: str) -> List[Dict]:
    url = "https://catalog.api.2gis.com/2.0/catalog/rubric/search"

    params = {
        'region_id': REGION_ID,
        'q': search_query,
        'key': API_KEY
    }

    try:
        response = requests.get(url, params=params, timeout=20)
        if response.status_code == 200:
            data = response.json()
            rubrics = data.get('result', {}).get('items', [])
            return rubrics
        else:
            print(f"Ошибка поиска рубрик'{search_query}': {response.status_code}")
    except Exception as e:
        print(f"Ошибка при поиске рубрик'{search_query}': {e}")

    return []

In [6]:
# Ищем инфраструктуру по рубрикам из инструкции по API
# Выбран радиус 1500м это примерно 15мин ходьбы

INFRASTRUCTURE_RUBRICS = {
    'метро': {
        'search_query': 'метро',
        'rubric_id': '535',  # Станции метро
        'radius': 1000
    },
    'кафе': {
        'search_query': 'кафе',
        'rubric_id': '161,165,162,112658,538,164',  # Кафе, рестораны, бары
        'radius': 1000
    },
    'торговый_центр': {
        'search_query': 'торговый центр',
        'rubric_id': '19499,611,110346',  # Торговые центры
        'radius': 1000
    },
    'супермаркет': {
        'search_query': 'супермаркет',
        'rubric_id': '350,9777,12127,112647',  # Супермаркеты
        'radius': 1000
    },
    'фитнес': {
        'search_query': 'тренажерный зал',
        'rubric_id': '268,643,267,20228,110427',  # Фитнес-центры
        'radius': 1000
    },
    'остановка': {
        'search_query': 'остановка',
        'rubric_id': '10792,113080,11081,422,416,9505,420,113081',  # Остановки транспорта
        'radius': 1000
    },
    'офис': {
        'search_query': 'офис',
        'rubric_id': '492,13796',  # Остановки транспорта
        'radius': 1000
    }
}


In [7]:
# Рассчет расстояний

import math

def calculate_distance(lat1: float, lon1: float, lat2: float, lon2: float) -> float:
    R = 6371000  # Радиус Земли в метрах

    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    a = math.sin(dlat/2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))

    distance_m = R * c
    return distance_m

In [8]:
# Функция для поиска инфраструктуры в рубриках

def search_infrastructure_by_rubric(lat: float, lon: float, infra_type: str) -> List[Dict]:
# Ищет объекты инфраструктуры по рубрикам в радиусе от координат

    if not lat or not lon:
        return []

    infra_set = INFRASTRUCTURE_RUBRICS.get(infra_type)
    if not infra_set:
        print(f"Неизвестный тип инфраструктуры: {infra_type}")
        return []

    rubric_id = infra_set['rubric_id']
    radius = infra_set['radius']

    params = {
        'key': API_KEY,
        'rubric_id': rubric_id,
        'point': f'{lon},{lat}',
        'radius': radius,
        'sort': 'distance',
        #'page_size': 5,
        'fields': 'items.id,items.name,items.point,items.address,items.rubrics'
    }

    try:
        response = requests.get(
            API_URL,
            params=params,
            timeout=15
        )

        if response.status_code == 200:
            data = response.json()
            объекты = data.get('result', {}).get('items', [])

            # Добавляем расстояние для каждого объекта
            for obj in объекты:
                точка = obj.get('point', {})
                obj_lat = точка.get('lat')
                obj_lon = точка.get('lon')

                if obj_lat and obj_lon:
                    # Добавляем координаты объектов инфраструктуры
                    obj['infra_lat'] = obj_lat
                    obj['infra_lon'] = obj_lon

                    # Добавляем адрес и названия
                    obj['infra_address'] = obj.get('address_name', '')
                    obj['infra_name'] = obj.get('name', '')

                    расстояние = calculate_distance(lat, lon, obj_lat, obj_lon)
                    obj['расстояние_м'] = round(расстояние)
                    obj['расстояние_км'] = round(расстояние / 1000, 2)
                    # Время пешком (5 км/ч = 1.4 м/с)
                    obj['время_пешком_мин'] = round((расстояние / 1.4) / 60, 1)

            return объекты
        else:
            print(f"Ошибка для {infra_type}: {response.status_code}")

    except Exception as e:
        print(f"Ошибка поиска {infra_type}: {e}")

    return []

In [9]:
# Функция для получения детальной информации об объекте и инфраструктуре вокруг

def get_details_with_infrastructure(object_id, lat=None, lon=None):

# items из инструкции по выбору
    detailed_fields = [
        'items.contacts',           # контакты
        'items.contact_groups',     # контакты компании (группы)
        'items.schedule',           # расписание работы
        'items.schedule_special',   # особое расписание
        'items.flags',              # признаки объекта
        'items.reviews',            # статистика по отзывам
        'items.rubrics',            # категории компании
        'items.org',                # организация (родительская)
        'items.brand',              # бренд
        'items.description',        # описание геообъекта
        'items.links',              # связанные объекты
        'items.name_ex',            # составные части наименования
        'items.statistics',         # сводная информация
        'items.caption',            # название объекта
        'items.congestion',         # загруженность филиала
        'items.access',             # тип доступа для парковки
        'items.access_comment',     # локализованное название для типа доступа
        'items.is_paid'            # является ли парковка платной
    ]

    params = {
        'id': object_id,
        'key': API_KEY,
        'fields': ','.join(detailed_fields)
    }

    try:
        response = requests.get(
            "https://catalog.api.2gis.ru/3.0/items/byid",
            params=params,
            timeout=20
        )

        if response.status_code == 200:
            data = response.json()
            details = data.get('result', {}).get('items', [])
            if details:
                result = details[0]

                # Если есть координаты, ищем инфраструктуру
                if lat and lon:
                    print(f"Ищем инфраструктуру вокруг объекта...")
                    infrastructure = {}

                    for infra_type in INFRASTRUCTURE_RUBRICS.keys():
                        print(f"Ищу {infra_type}...", end=" ")
                        found_infrt = search_infrastructure_by_rubric(lat, lon, infra_type)

                        if found_infrt:
                            infrastructure[infra_type] = found_infrt
                            print(f"Найдено {len(found_infrt)}")
                        else:
                            print("Не найдено")

                        # Пауза между запросами
                        time.sleep(0.5)

                    result['infrastructure'] = infrastructure

                    # Добавляем статистику по инфраструктуре
                    infrastructure_stats = {}
                    infra_total = 0

                    for category, items in infrastructure.items():
                        infrastructure_stats[f'{category}_count'] = len(items)
                        infra_total += len(items)

                        if items:
                            # Находим ближайший объект в категории
                            distance = [item.get('расстояние_м', 999999) for item in items]
                            valid_distance = [d for d in distance if isinstance(d, (int, float))]

                            if valid_distance:
                                min_distance = min(valid_distance)
                                infrastructure_stats[f'{category}_nearest_m'] = min_distance
                                infrastructure_stats[f'{category}_nearest_km'] = round(min_distance / 1000, 2)

                                # Добавим кооридинаты и названия
                                nearest_obj = items[0]
                                infrastructure_stats[f'{category}_nearest_lat'] = nearest_obj.get('infra_lat')
                                infrastructure_stats[f'{category}_nearest_lon'] = nearest_obj.get('infra_lon')
                                infrastructure_stats[f'{category}_nearest_name'] = nearest_obj.get('infra_name', nearest_obj.get('name', ''))
                                infrastructure_stats[f'{category}_nearest_address'] = nearest_obj.get('infra_address', '')

                    result['infrastructure_stats'] = infrastructure_stats
                    result['infrastructure_total'] = infra_total

                return result

    except Exception as e:
        print(f"Ошибка для {object_id}: {e}")

    return None

In [ ]:
# Главный импорт - запуск поиска по всем ключевым словам

import requests
import time
import json
from collections import defaultdict

all_objects = []
keyword_stats = defaultdict(list)

for i, keyword in enumerate(keywords, 1):
    print(f"\n[{i}/{len(keywords)}] Поиск: '{keyword}'")

    objects = search_objects(keyword)

    print(f"Найдено объектов: {len(objects)}")

    # Сохраняем ВСЕ объекты (с дубликатами)
    for obj in objects:
        obj_id = obj.get('id')
        if obj_id:
            keyword_stats[keyword].append(obj_id)

            # Добавляем информацию о запросе
            obj['search_keyword'] = keyword
            obj['request_number'] = i
            all_objects.append(obj)

    # Пауза между запросами, можно и чутка больше
    time.sleep(1.5)

print(f"ПОИСК ЗАВЕРШЕН!")

In [ ]:
# Получаем всю детальную информацию с инфраструктурой

print("\nПолучаем всю детальную информацию с инфраструктурой...")

objects_with_details = []

objects_to_process = all_objects # когда подключу платный API, то можно будет протестировтаь на нескольких объектах
print(f"Обработаем {len(objects_to_process)} объектов")

for i, obj in enumerate(objects_to_process, 1):
    obj_id = obj.get('id')
    obj_name = obj.get('name', 'Без названия')

    # Получаем координаты из объекта
    lat, lon = None, None
    if 'point' in obj and 'lat' in obj['point'] and 'lon' in obj['point']:
        lat = obj['point']['lat']
        lon = obj['point']['lon']

    print(f"\n[{i}/{len(objects_to_process)}] {obj_name[:40]}...")
    print(f" Координаты: {lat}, {lon}")

    if obj_id and lat and lon:
        details = get_details_with_infrastructure(obj_id, lat, lon)

        if details:
            # Объединяем данные
            full_object = {**obj, **details}
            objects_with_details.append(full_object)

            # Показываем статистику
            infra_count = full_object.get('infrastructure_total', 0)
            print(f"{infra_count} объектов")
        else:
            print(f"Не удалось получить детали")
    else:
        print(f"Нет ID или координат")

    # Пауза для API
    if i % 3 == 0:
        time.sleep(1)

In [18]:
# Анализ инфраструктуры

from typing import List, Dict, Any

def analyze_infrastructure(objects: List[Dict]) -> Dict[str, Any]:

    analysis = {
        'total_objects': len(objects),
        'infrastructure_summary': {},
        'best_objects': []
    }

    # Собираем статистику по всем объектам
    for obj in objects:
        if 'infrastructure_stats' in obj:
            stats = obj['infrastructure_stats']

            for key, value in stats.items():
                # Включаем только числовые поля - до этого выдавало ошибку, если залетали строки
                if any(x in key for x in ['_count', '_nearest_m', '_nearest_km']):
                    if key not in analysis['infrastructure_summary']:
                        analysis['infrastructure_summary'][key] = []
                    analysis['infrastructure_summary'][key].append(value)

    # Рассчитываем средние значения ТОЛЬКО для числовых полей
    avg_stats = {}
    for key, values in analysis['infrastructure_summary'].items():
        # Фильтруем None значения
        valid_values = [v for v in values if v is not None]
        if valid_values:
            # Проверяем что все значения числовые
            numeric_values = []
            for v in valid_values:
                try:
                    numeric_values.append(float(v))
                except (ValueError, TypeError):
                    continue

            if numeric_values:
                avg_stats[f'avg_{key}'] = sum(numeric_values) / len(numeric_values)
                avg_stats[f'min_{key}'] = min(numeric_values)
                avg_stats[f'max_{key}'] = max(numeric_values)

    analysis['average_stats'] = avg_stats

    # Находим объекты с лучшей инфраструктурой
    for obj in objects:
        if 'infrastructure_stats' in obj:
            score = 0
            stats = obj['infrastructure_stats']

            # 1 - Критерий: есть ли метро рядом (самый важный)
            if 'метро_nearest_m' in stats and stats['метро_nearest_m']:
                distance = stats['метро_nearest_m']
                if distance <= 500:  # до 500м - отлично
                    score += 50
                elif distance <= 1000:  # до 1км - хорошо
                    score += 30
                elif distance <= 1500:  # до 1.5км - удовлетворительно
                    score += 15

            # 2 - Критерий: есть ли парковка рядом
            if 'парковка_count' in stats and stats['парковка_count'] > 0:
                score += 20

            # 3 - Критерий: доступность питания
            if 'кафе_count' in stats:
                cafe_count = stats['кафе_count']
                if cafe_count >= 3:
                    score += 15
                elif cafe_count >= 1:
                    score += 8

            # 4 - Критерий: доступность магазинов
            if 'супермаркет_count' in stats:
                market_count = stats['супермаркет_count']
                if market_count >= 2:
                    score += 10
                elif market_count >= 1:
                    score += 5

            # 5 - Критерий: другие важные объекты
            important_objects = ['банк', 'аптека', 'торговый_центр']
            for obj_type in important_objects:
                if f'{obj_type}_count' in stats and stats[f'{obj_type}_count'] > 0:
                    score += 5

            # 6 - Критерий: общее количество объектов инфраструктуры
            total_infra = obj.get('infrastructure_total', 0)
            if total_infra >= 20:
                score += 20
            elif total_infra >= 15:
                score += 15
            elif total_infra >= 10:
                score += 10
            elif total_infra >= 5:
                score += 5

            analysis['best_objects'].append({
                'name': obj.get('name', 'Без названия'),
                'address': obj.get('address_name', 'Нет адреса'),
                'score': score,
                'infrastructure_total': total_infra,
                'stats': stats
            })

    # Сортируем по оценке
    analysis['best_objects'].sort(key=lambda x: x['score'], reverse=True)

    return analysis

In [19]:
# Анализируем дальше и разбиваем по деталям

if objects_with_details:
    print("\nАНАЛИЗ ИНФРАСТРУКТУРЫ:")

    infrastructure_analysis = analyze_infrastructure(objects_with_details)

    print(f"Проанализировано объектов: {infrastructure_analysis['total_objects']}")

    # Показываем средние показатели
    if 'average_stats' in infrastructure_analysis:
        print("\nСредние показатели:")
        for key, value in infrastructure_analysis['average_stats'].items():
            if '_count' in key:
                print(f"  {key.replace('avg_', '').replace('_count', '')}: {value:.1f}")
            elif '_nearest_m' in key:
                print(f"  {key.replace('avg_', '').replace('_nearest_m', '')}: {value:.0f} м")

    # Считаем объекты с хорошей инфраструктурой
    good_threshold = 50 # задала 50 как порог для оценки хорошей инфраструктуры
    objects_with_good_infra = sum(1 for obj in infrastructure_analysis['best_objects']
                                  if obj['score'] >= good_threshold)

    print(f"Объектов с хорошей инфраструктурой (оценка ≥ {good_threshold}): {objects_with_good_infra}")
    print(f"Процент объектов с хорошей инфраструктурой: {(objects_with_good_infra/len(objects_with_details)*100):.1f}%")


АНАЛИЗ ИНФРАСТРУКТУРЫ:
Проанализировано объектов: 245

Средние показатели:

Объектов с хорошей инфраструктурой (оценка ≥ 50): 0
Процент объектов с хорошей инфраструктурой: 0.0%


In [ ]:
import pandas as pd
from datetime import datetime


if objects_with_details:
    try:
        # Создаем список для всех данных
        all_rows = []

        for sport_obj in objects_with_details:
            # Основные данные спортивного объекта
            sport_object_data = {
                'sport_object_id': sport_obj.get('id'),
                'sport_object_name': sport_obj.get('name', ''),
                'sport_object_address': sport_obj.get('address_name', sport_obj.get('address', '')),
                'sport_object_lat': sport_obj.get('point', {}).get('lat') if isinstance(sport_obj.get('point'), dict) else '',
                'sport_object_lon': sport_obj.get('point', {}).get('lon') if isinstance(sport_obj.get('point'), dict) else '',
                'sport_object_type': 'теннис' if 'теннис' in str(sport_obj.get('name', '')).lower() else
                                    'сквош' if 'сквош' in str(sport_obj.get('name', '')).lower() else
                                    'паддл' if any(w in str(sport_obj.get('name', '')).lower() for w in ['паддл', 'падл', 'padel', 'падел', 'паддел']) else
                                    'другое',
                'search_keyword': sport_obj.get('search_keyword', ''),
                'sport_object_total_infrastructure': sport_obj.get('infrastructure_total', 0)
            }

            # Добавляем контактную информацию
            contacts = sport_obj.get('contacts', [])
            if contacts:
                phones = []
                emails = []
                websites = []

                for contact in contacts:
                    if contact.get('type') == 'phone':
                        phones.append(contact.get('value', ''))
                    elif contact.get('type') == 'email':
                        emails.append(contact.get('value', ''))
                    elif contact.get('type') == 'website':
                        websites.append(contact.get('value', ''))

                sport_object_data['phones'] = ' | '.join(phones) if phones else ''
                sport_object_data['emails'] = ' | '.join(emails) if emails else ''
                sport_object_data['websites'] = ' | '.join(websites) if websites else ''

            # Добавляем расписание
            if 'schedule' in sport_obj and sport_obj['schedule']:
                schedule_str = json.dumps(sport_obj['schedule'], ensure_ascii=False)
                sport_object_data['schedule'] = schedule_str

            # Добавляем описание
            if 'description' in sport_obj and sport_obj['description']:
                sport_object_data['description'] = sport_obj['description']

            # Добавляем рубрики
            if 'rubrics' in sport_obj and sport_obj['rubrics']:
                rubrics = [r.get('name', '') for r in sport_obj['rubrics'] if r.get('name')]
                sport_object_data['rubrics'] = ' | '.join(rubrics)

            # Если есть инфраструктура, создаем строки для КАЖДОГО объекта инфраструктуры
            if 'infrastructure' in sport_obj and sport_obj['infrastructure']:
                for infra_type, infra_objects in sport_obj['infrastructure'].items():
                    for infra_obj in infra_objects:
                        # Данные объекта инфраструктуры
                        infra_data = {
                            'infrastructure_type': infra_type,
                            'infrastructure_name': infra_obj.get('infra_name', infra_obj.get('name', '')),
                            'infrastructure_address': infra_obj.get('infra_address', infra_obj.get('address_name', infra_obj.get('address', ''))),
                            'infrastructure_lat': infra_obj.get('infra_lat', infra_obj.get('point', {}).get('lat')),
                            'infrastructure_lon': infra_obj.get('infra_lon', infra_obj.get('point', {}).get('lon')),
                            'distance_meters': infra_obj.get('расстояние_м', ''),
                            'distance_kilometers': infra_obj.get('расстояние_км', ''),
                            'walk_time_minutes': infra_obj.get('время_пешком_мин', ''),
                            'infrastructure_id': infra_obj.get('id', '')
                        }

                        # Объединяем данные спортивного объекта и инфраструктуры
                        row = {**sport_object_data, **infra_data}
                        all_rows.append(row)

            # Если НЕТ инфраструктуры, добавляем только спортивный объект -- но по идее таких нет
            else:
                # Добавляем пустые поля для инфраструктуры
                infra_data = {
                    'infrastructure_type': '',
                    'infrastructure_name': '',
                    'infrastructure_address': '',
                    'infrastructure_lat': '',
                    'infrastructure_lon': '',
                    'distance_meters': '',
                    'distance_kilometers': '',
                    'walk_time_minutes': '',
                    'infrastructure_id': ''
                }
                row = {**sport_object_data, **infra_data}
                all_rows.append(row)

        # Создаем ДФ
        df = pd.DataFrame(all_rows)

        # Сохраняем в CSV
        csv_filename = "all_sport_objects_with_infrastructure.csv"
        df.to_csv(csv_filename, index=False, encoding='utf-8-sig')

    except Exception as e:
        print(f"Ошибка: {e}")
else:
    print("Нет данных")